In [1]:
import geopandas as gpd
import pandas as pd
import requests
import os
import zipfile
import rasterio
from rasterio.merge import merge
from tqdm import tqdm
import pyproj
# !pip install pyrosm
os.environ['PROJ_LIB'] =pyproj.datadir.get_data_dir()

import timeit
import osmnx as ox
from pyrosm import get_data

import osmium
import shapely.wkb as wkblib
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd
import rasterio
from rasterio.features import geometry_mask
from rasterio.crs import CRS
from tqdm import tqdm

In [ ]:
!pip install --upgrade geopandas

In [ ]:
import os
os.environ["PROJ_LIB"] = "C:/Users/Ondrej/.conda/envs/ssml/Lib/site-packages/pyproj"

In [ ]:
print(pyproj.datadir)

In [ ]:
os.environ['PROJ_LIB'] =pyproj.datadir.get_data_dir()

# Street data

### Cities

In [ ]:
region = "Amsterdam"
G = ox.graph_from_place(region, network_type="all") # get all networks for the given region
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G) # convert the graph to a GeoDataFrame

# reproject to the matching projection with dtm and dsm
gdf_nodes = gdf_nodes.to_crs('EPSG:28992')
gdf_edges = gdf_edges.to_crs('EPSG:28992')

In [ ]:
cities = ["Amsterdam, Netherlands", "Rotterdam, Netherlands", "Utrecht, Netherlands", "The Hague, Netherlands"]

all_gdf_nodes = []
all_gdf_edges = []

# Iterate over cities with progress tracking
total_cities = len(cities)
for i, city in enumerate(cities):
    G = ox.graph_from_place(city, network_type="all")
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

    gdf_nodes = gdf_nodes.to_crs('EPSG:28992') # reproject to the matching projection with dtm and dsm
    gdf_edges = gdf_edges.to_crs('EPSG:28992')

    all_gdf_nodes.append(gdf_nodes)
    all_gdf_edges.append(gdf_edges)
    # progress
    progress = f"{i+1}/{total_cities}"
    print(f"Processing city {city} ({progress})")

# merge into a geodataframe
merged_gdf_nodes = gpd.GeoDataFrame(pd.concat(all_gdf_nodes, ignore_index=True))
merged_gdf_edges = gpd.GeoDataFrame(pd.concat(all_gdf_edges, ignore_index=True))

In [ ]:
# filter down to only geometries
edges_geometry = merged_gdf_edges[["geometry"]]

# save the file as gpkg
edges_geometry.to_file('cities_streets.gpkg', driver='GPKG')

### Entire Netherlands

In [ ]:
save_dir = "C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands"
fp = get_data("Netherlands", directory=save_dir)
print(fp)

In [ ]:
class StreetsHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.num_nodes = 0
        self.num_relations = 0
        self.num_ways = 0
        self.street_relations = []
        self.street_relation_members = []
        self.street_ways = []
        # A global factory that creates WKB from a osmium geometry
        self.wkbfab = osmium.geom.WKBFactory()

    def way(self, w):
        if w.tags.get("highway") is not None and w.tags.get("name") is not None:
            try:
                wkb = self.wkbfab.create_linestring(w)
                geo = wkblib.loads(wkb, hex=True)
            except:
                return
            row = { "w_id": w.id, "geo": geo }

            for key, value in w.tags:
                row[key] = value

            self.street_ways.append(row)
            self.num_ways += 1

    def relation(self, r):
        if r.tags.get("type") == "associatedStreet" and r.tags.get("name") is not None:
            row = { "r_id": r.id }
            for key, value in r.tags:
                row[key] = value
            self.street_relations.append(row)

            for member in r.members:
                self.street_relation_members.append({ 
                    "r_id": r.id, 
                    "ref": member.ref, 
                    "role": member.role, 
                    "type": member.type, })
                self.num_relations += 1

In [ ]:
handler = StreetsHandler()

# start data file processing
handler.apply_file("/street_data/netherlands-latest.osm.pbf", locations=True, idx='flex_mem')
street_ways_df = pd.DataFrame(handler.street_ways)

# filter data to relevant coluns
street_ways_df_filtered = street_ways_df[['w_id','geo','highway','name']]
street_ways_df_filtered

In [ ]:
# check counts of groups of roads
sum(street_ways_df_filtered['highway'] == 'pedestrian')
street_ways_df_filtered['highway'].value_counts()

In [ ]:
# save data 
gdf = gpd.GeoDataFrame(street_ways_df_filtered, geometry='geo')
gdf.to_file('netherlands_streets.gpkg', driver='GPKG')

# DSM/DTM files from AHN3

The below download DSM/DTM tiles and merges them into a single .tif file. The input url links for the input text file can be found from the following website:https://app.pdok.nl/rws/ahn3/download-page/#, where tile url links look as follows: https://download.pdok.nl/rws/ahn3/v1_0/05m_dsm/R_02EZ1.ZIP

In [ ]:
import os
import requests
import os
import rasterio

def download_tiles(input_directory, working_dir):
    # Read tile URLs from the input directory
    with open(input_directory, 'r') as file:
        tile_urls = file.read().splitlines()

    os.makedirs(working_dir, exist_ok=True)
    # Progress
    total_tiles = len(tile_urls) 
    downloaded_tiles = 0

    for url in tile_urls:
        file_name = url.split('/')[-1]
        output_path = os.path.join(working_dir, file_name)
        response = requests.get(url)
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print("Downloaded", file_name)
        print("Saved to", os.path.abspath(output_path))

        # Update the progress
        downloaded_tiles += 1
        print("Progress: {}/{}".format(downloaded_tiles, total_tiles))

        
def extract_files(working_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)  # Create the 'extracted' directory

    for file in os.listdir(working_directory):
        if zipfile.is_zipfile(os.path.join(working_directory, file)):
            with zipfile.ZipFile(os.path.join(working_directory, file)) as item:
                item.extractall(output_directory)
                print("Extracted files from:", file)

    print("Files extracted and saved to:", output_directory)
    


def merge_tif_files(extracted_directory, output_directory, filename):
    os.makedirs(output_directory, exist_ok=True)

    # get tiles into a list
    tif_files = []
    for root, dirs, files in os.walk(extracted_directory):
        tif_files += [os.path.join(root, file) for file in files if file.endswith('.TIF')]

    # merge files
    merged_arr, out_trans = merge(tif_files)

    # Output file path
    output_file = os.path.join(output_directory, f'{filename} merged.tif')

    # Output and save the file
    with rasterio.open(tif_files[0]) as src:
        meta = src.meta

    meta.update({"driver": "GTiff",
                 "height": merged_arr.shape[1],
                 "width": merged_arr.shape[2],
                 "transform": out_trans})

    with rasterio.open(output_file, "w", **meta) as dst:
        dst.write(merged_arr)
    
input_dir = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_list_dsm.txt'
working_dir = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_dtm'
output_dir = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_dtm/extracted'

# download_tiles(input_dir, working_dir)
# extract_files(working_dir, output_dir)
merge_tif_files(output_dir, working_dir, 'DSM')

In [ ]:
working_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_dtm'
output_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/extracted'

os.makedirs(output_directory, exist_ok=True)  # Create the 'extracted' directory

for file in os.listdir(working_directory):
    if zipfile.is_zipfile(os.path.join(working_directory, file)):
        with zipfile.ZipFile(os.path.join(working_directory, file)) as item:
            item.extractall(output_directory)
            print("Extracted files from:", file)

print("Files extracted and saved to:", output_directory)

In [ ]:
extracted_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/extracted'
output_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection'
output_file = os.path.join(output_directory, 'merged.tif')

os.makedirs(output_directory, exist_ok=True) 

# get all the tiles into a list
tif_files = []
for root, dirs, files in os.walk(extracted_directory):
    tif_files += [os.path.join(root, file) for file in files if file.endswith('.TIF')]

# total
total_files = len(tif_files)
print(total_files)

# merge the files
merged_arr, out_trans = merge(tif_files)

# output and save the file 
with rasterio.open(tif_files[0]) as src:
    meta = src.meta

meta.update({"driver": "GTiff",
             "height": merged_arr.shape[1],
             "width": merged_arr.shape[2],
             "transform": out_trans})

with rasterio.open(output_file, "w", **meta) as dst:
    dst.write(merged_arr)

# Missing values

In [2]:
# load street file for the entire netherlands 
streets_file = 'street_data/netherlands_streets.gpkg'
streets = gpd.read_file(streets_file)
streets = streets.to_crs('EPSG:28992')

In [3]:
os.makedirs('batch_results/', exist_ok=True)

# get the list of tiles in numerical order 
tile_files = [filename for filename in os.listdir('/tiles/DSM/DSM_cropped/') if filename.endswith('.tif')]
tile_files.sort(key=lambda x: int(os.path.splitext(x)[0]) if os.path.splitext(x)[0].isdigit() else float('inf'))

# number of batches based on batch size 
batch_size = 100
total_batches = len(tile_files) // batch_size + 1


# for batch_idx in tqdm(range(0, len(tile_files), batch_size), desc='Processing Batches'):
for batch_idx in tqdm(range(100, 200, batch_size), desc='Processing Batches'):
    batch_files = tile_files[batch_idx:batch_idx + batch_size]
    
    results_df = pd.DataFrame(columns=['tile', 'masked_values', 'missing_values', 'non_missing_values', 'missing_values_perc'])

    for filename in tqdm(batch_files, desc='Processing Tiles'):
        dsm_file = os.path.join(tiles_folder, filename) # tiles paths 
        tile_name = os.path.splitext(filename)[0]
        print(tile_name)
        
        # open the indivisual DSM files
        with rasterio.open(dsm_file, 'r+') as src:
            target_crs = CRS.from_epsg(28992)
            dsm_extent = src.bounds
            src.crs = target_crs
            crs = src.crs
            dsm_data = src.read(1)
            
            # filter the streets based on the selected DSM file's extent
            streets2 = streets.cx[dsm_extent[0]:dsm_extent[2], dsm_extent[1]:dsm_extent[3]]
            
            # create a polygon of DSM extent to remove overreaching streets
            dsm_extent_polygon = Polygon([(dsm_extent[0], dsm_extent[1]), (dsm_extent[0], dsm_extent[3]),
                                          (dsm_extent[2], dsm_extent[3]), (dsm_extent[2], dsm_extent[1])])
            
            # filter the streets based on the polygon's extent 
            streets2 = streets2[streets2.geometry.within(dsm_extent_polygon)]
            streets2 = streets2['geometry']
            
            # buffer around the streets
            buffer_distance = 300
            streets_union = streets2.geometry.unary_union
            
            # create an exception if a dsm tile has only nans (like ocean present)
            try:    
                buffered_streets = streets_union.buffer(buffer_distance)
                
                # rasterize the buffered streets
                out_shape = dsm_data.shape
                transform = src.transform
                buffer_mask = geometry_mask(
                    [buffered_streets],
                    out_shape=out_shape,
                    transform=transform,
                    invert=True,
                    all_touched=True,
                )
                
                # mask TRUE/FALSE inside/outside the buffer
                mask = buffer_mask.astype(bool)
                
                # Count the total number of values & NaN and non-NaN values within the mask
                num_true = np.count_nonzero(mask)
                num_nan = np.count_nonzero(np.isnan(dsm_data[mask]))
                num_non_nan = np.count_nonzero(~np.isnan(dsm_data[mask]))
        
                # calculate the percentage of missing values inside of the buffer
                missing_percentage = num_nan / (num_nan + num_non_nan) * 100

                # add values to the dataframe 
                new_row = pd.DataFrame({'tile': [tile_name], 'masked_values': [num_true], 'missing_values': [num_nan], 'non_missing_values': [num_non_nan], 'missing_values_perc': [missing_percentage]})
                results_df = pd.concat([results_df, new_row], ignore_index=True)

            except AttributeError:
                pass
    
    # save after each batch iteration 
    output_csv = os.path.join(output_folder, f'tiles_batch{batch_idx // batch_size + 1}.csv')
    results_df.to_csv(output_csv, index=False)

In [20]:
results_df.head()

,tile,masked_values,missing_values,non_missing_values,missing_values_perc
0,31,1061952,338875,723077,31.910576
1,32,3214552,334132,2880420,10.394357
2,33,3368922,27374,3341548,0.812545
3,34,3416190,11133,3405057,0.325889
4,35,3362819,34328,3328491,1.020810
